In [16]:
import os
# Find the latest version of spark 3.0 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.1.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:2 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:3 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:4 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:6 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:9 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [83.3 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:12 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Fetched 261 kB in 3s (83.8 kB/s)
Reading package li

In [ ]:
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar


--2022-11-05 15:29:58--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar.2’

postgresql-42.2.9.j 100%[===================>] 892.61K  5.80MB/s    in 0.2s    

2022-11-05 15:29:58 (5.80 MB/s) - ‘postgresql-42.2.9.jar.2’ saved [914037/914037]



In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("BigData").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

In [ ]:
from pyspark import SparkFiles
# Load in employee.csv from S3 into a DataFrame
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Wireless_v1_00.tsv.gz"
spark.sparkContext.addFile(url)



In [ ]:
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get(""), sep="\t", header=True, inferSchema=True)
df.show(10)

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   16414143|R3W4P9UBGNGH1U|B00YL0EKWE|     852431543|LG G4 Case Hard T...|        Wireless|          2|            1|          3|   N|                Y|Looks good, funct...|2 issues  -  Once...| 2015-08-31|
|         US|   50800750|R15V54KBMTQWAY|B00XK95RPQ|     516894650|Selfie Stick Fibl...|        Wireless|          4|    

**Transform**

In [ ]:
# Count the number of records (rows) in the dataset.


print(df.count())


9002021


In [27]:
#dropping null values 
df = df.dropna()


In [28]:
#dropping duplicate rows 
df = df.dropDuplicates()


In [29]:
# Count the number of records (rows) in the dataset again after the above cleaning
print(df.count())

9001052


In [30]:
df.printSchema() 

root
 |-- marketplace: string (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- review_id: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: integer (nullable = true)
 |-- product_title: string (nullable = true)
 |-- product_category: string (nullable = true)
 |-- star_rating: string (nullable = true)
 |-- helpful_votes: integer (nullable = true)
 |-- total_votes: integer (nullable = true)
 |-- vine: string (nullable = true)
 |-- verified_purchase: string (nullable = true)
 |-- review_headline: string (nullable = true)
 |-- review_body: string (nullable = true)
 |-- review_date: string (nullable = true)



In [31]:
review_id_table = df.select(["review_id", "customer_id", "product_id", "product_parent", "review_date"]).drop_duplicates()
review_id_table.show(5)

+--------------+-----------+----------+--------------+-----------+
|     review_id|customer_id|product_id|product_parent|review_date|
+--------------+-----------+----------+--------------+-----------+
|R10J4QVZO9RYGX|   52806896|B0089N9WLS|     100481075| 2014-01-08|
|R10PMRAOP777DF|    5735770|B009RXWL5S|       1320762| 2013-12-21|
|R138RZH9Y9CXLV|    4539797|B00F9DPHJ2|     172801534| 2014-07-08|
|R13WKG9EKOAP3O|   47165262|B00CHV2ULC|      82728480| 2014-02-24|
|R142ARXUBFE16F|   10498660|B00OMFPA7M|     828867330| 2015-03-01|
+--------------+-----------+----------+--------------+-----------+
only showing top 5 rows



In [57]:
products = df.select(["product_id", "product_title"]).drop_duplicates()


In [33]:
vine_table = df.select(["review_id", "star_rating", "helpful_votes", "total_votes", "vine"]).drop_duplicates()

In [34]:
customers = df.select(["customer_id"]).groupBy('customer_id').count().orderBy('count')

In [46]:
customers.printSchema() 

root
 |-- customer_id: integer (nullable = true)
 |-- count: long (nullable = false)



In [47]:
# previous cell shows 'count' column above - need to rename this to customer_count 
customers = customers.withColumnRenamed('count','customer_count')


In [49]:
#check schema again 
customers.printSchema() 

root
 |-- customer_id: integer (nullable = true)
 |-- customer_count: long (nullable = false)



In [35]:
#Need to transform the dataset to fit the tables in the schema file
from pyspark.sql.types import DateType

review_id_table = review_id_table.withColumn('review_date',df['review_date'].cast(DateType()))


In [36]:
#print schema to check 
review_id_table.printSchema() 

root
 |-- review_id: string (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: integer (nullable = true)
 |-- review_date: date (nullable = true)



In [37]:
 # Need to cast star_rating as integer 
from pyspark.sql.types import IntegerType

vine_table = vine_table.withColumn("star_rating",df["star_rating"].cast(IntegerType()))


In [38]:
#print schema to check 
vine_table.printSchema() 

root
 |-- review_id: string (nullable = true)
 |-- star_rating: integer (nullable = true)
 |-- helpful_votes: integer (nullable = true)
 |-- total_votes: integer (nullable = true)
 |-- vine: string (nullable = true)



**Load**

In [ ]:
from google.colab import files
files.upload()

In [40]:
from rds_config import password

In [41]:
 # Configuration for RDS instance
mode="append"
jdbc_url = "jdbc:postgresql://<endpoint>:5432/big_data_hwk2"
config = {"user":"postgres",
          "password": password,
          "driver":"org.postgresql.Driver"}

In [ ]:
# Write DataFrame to table

products.write.jdbc(url=jdbc_url, table='products', mode=mode, properties=config)

#this cell had already successfully ran, the error was a keyboard interrupt as it takes a long time to run again!

In [53]:
# Write DataFrame to table

customers.write.jdbc(url=jdbc_url, table='customers', mode=mode, properties=config)

In [44]:
# Write DataFrame to table

vine_table.write.jdbc(url=jdbc_url, table='vine_table', mode=mode, properties=config)

In [54]:
# Write DataFrame to table

review_id_table.write.jdbc(url=jdbc_url, table='review_id_table', mode=mode, properties=config)